In [9]:
import requests
import json
import re
from bs4 import BeautifulSoup

from webdriver_manager.microsoft import EdgeChromiumDriverManager
from selenium.webdriver.edge.service import Service as EdgeService
from selenium.webdriver import EdgeOptions


from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver import ChromeOptions

from webdriver_manager.firefox import GeckoDriverManager
from selenium.webdriver.firefox.service import Service as FirefoxService
from selenium.webdriver import FirefoxOptions

from selenium import webdriver
from selenium.webdriver.common.by import By


import webbrowser
import time

import tkinter as tk
from tkinter import filedialog
from os import mkdir



home_url = "https://animepahe.ru/"
anime_url = home_url+"anime/"
api_url_extension = "api?m="
search_url_extension = api_url_extension+"search&q="
quality = "720p"
sub_or_dub = "sub"

#keyword = input("Enter the name of the anime you want to download> ")
keyword = "Senyu"

#Searches for the anime in the animepahe database
def Searcher(keyword):
    full_search_url = home_url+search_url_extension+keyword
    response = requests.get(full_search_url)
    results = json.loads(response.content.decode("UTF-8"))["data"]
    return results

results = Searcher(keyword)

In [10]:
#Prompts the user to select the index of the anime they want from a list of the search results and returns the id of the chosen anime
def AnimeSelection(results):
    while "anime_id" not in locals():    
        print("Please enter the number belonging to the anime you want from the list below")
        for index, result in enumerate(results):
            print(index+1, result["title"])

        index_of_chosen_anime = int(input("Number> "))-1

        if not index_of_chosen_anime < 0 and not len(results) <= index_of_chosen_anime:
            for index, result in enumerate(results):
                if index == index_of_chosen_anime:
                    anime_id = result["session"]
                    anime_title = result["title"]
                    return anime_id, anime_title
        else:
            print("\nInvalid number Senpwai")

anime_id, anime_title = AnimeSelection(results)




Please enter the number belonging to the anime you want from the list below
1 Senyuu.
2 Senyuu. 2
3 Senyuu. Specials
4 Senjuushi
5 Senryuu Shoujo
6 Senyoku no Sigrdrifa
7 Senyoku no Sigrdrifa: Dai-909 Senjutsu Hime-tai Sentou Kiroku!
8 Chiisai Sensuikan ni Koi wo Shita Dekasugiru Kujira no Hanashi


In [11]:
def SetDownloadFolderPath(anime_title):
    
    root = tk.Tk()
    root.withdraw()

# Prompt the user to choose a folder directory
    folder_path = filedialog.askdirectory(title="Choose folder to put the downloaded anime")
    download_folder = folder_path+"/"+anime_title
    download_folder = download_folder.replace("/", "\\\\")
    mkdir(download_folder)
    return download_folder



#download_folder_path = SetDownloadFolderPath(anime_title)
download_folder_path = "C:\\Anime\\"+anime_title
mkdir(download_folder_path)

In [12]:

#Issues a GET command to the server together with the id of the anime and returns a list of the links(not donwload links) to all the episodes of that anime
def EpisodeLinks(anime_id):
    response = requests.get(home_url+api_url_extension+"release&id="+anime_id+"&sort=episode_asc")
    episode_data = json.loads(response.content.decode("UTF-8"))["data"]
    episode_sessions = [episode["session"] for episode in episode_data]
    episode_links = [home_url+"play/"+anime_id+"/"+str(episode_session) for episode_session in episode_sessions]
    return episode_links

episode_links = EpisodeLinks(anime_id)

In [13]:
def DownloadData(episode_links):
    download_data = []
    for episode_link in episode_links:
        episode_page = requests.get(episode_link).content
        soup = BeautifulSoup(episode_page, "html.parser")
        download_data.append(soup.find_all("a", class_="dropdown-item", target="_blank"))
    #Scrapes the download data of each episode and stores the links for each quality and dub or sub in a list which is contained in another list containing all episodes
    download_links = [[download_link["href"] for download_link in episode_data] for episode_data in download_data]
    #Scrapes the download data of each episode and stores the info for each quality and dub or sub in a list which is contained in another list containing all episodes
    download_info = [[episode_info.text.strip() for episode_info in episode_data] for episode_data in download_data] 
    return (download_links, download_info)   
download_links, download_info = DownloadData(episode_links)


In [14]:
#Returns the sizes of the downloads
def DownloadSizes(download_info):
    #You dont have to understand what happens here cause even I don't lol, ChatGpt for the win
    #The function uses a regular expression to find an expression that matches it's "syntax" starts with ( and ends with MB)
    #This match is then striped of the MB part in order to convert it into an integer, but before that it is converted into a string cause re.findall returns a string
    #The string is then converted into an integer and voila we have the sizes of our episodes
    download_sizes = [[int("".join(re.findall(r'\((.*?)MB\)'.strip("MB"), episode_info))) for episode_info in episode_data] for episode_data in download_info]
    return download_sizes

download_sizes = DownloadSizes(download_info)
    


In [15]:
#Prompts the user for the settings of their downloads, the quality and whether subbed or dubbed

def DownloadSettings():

    met_conditions = 0
    def quality_chooser(quality):
        if quality == "360p" or quality == "360":
            return 0
        elif quality == "720p" or quality =="720":
            return 1
        elif quality == "1080p" or quality =="1080":
            return 2
        else:
            return "error"
    
    def sub_or_dubber(sub_or_dub):
        if sub_or_dub == "sub" or sub_or_dub == "s":
            return 1
        elif sub_or_dub == "dub" or sub_or_dub == "dub":
            return 0
        else:
            return "error"

    
    quality = 69
    sub_or_dub = 69

    while met_conditions < 2:
        while quality_chooser(quality) == "error":
            quality = input("What quality do you want to download in Senpwai uWu? 360p, 720p or 1080p> ")
        quality = quality_chooser(quality)
        met_conditions+=1

        while sub_or_dubber(sub_or_dub) == "error":
            sub_or_dub = input("Sub or dub> ").lower()
        sub_or_dub = sub_or_dubber(sub_or_dub)
        met_conditions+=1

    return quality, sub_or_dub


quality, sub_or_dub = DownloadSettings()

    

In [16]:
#Generates new download links and download sizes for the episodes based off the user's configured settings
def ConfigureDownloadData(download_links, download_sizes, quality, sub_or_dub):
    if sub_or_dub:
        print("sub")
        configured_download_links = [episode_links[:3] for episode_links in download_links]
        configured_download_sizes = [episode_links[:3] for episode_links in download_sizes]
    elif not sub_or_dub:
        if len(download_links[0]) == 3:
            print("There seems to be no dub for this anime, switching to sub")
            configured_download_links = [episode_links[:3] for episode_links in download_links]
            configured_download_sizes = [episode_links[:3] for episode_links in download_sizes]
        elif len(download_links[0]) == 6:
            print("dub")
            configured_download_links = [episode_links[3:] for episode_links in download_links]
            configured_download_sizes = [episode_links[3:] for episode_links in download_sizes]

    configured_download_links = [episode_links[quality] for episode_links in configured_download_links]
    configured_download_sizes = [episode_links[quality] for episode_links in configured_download_sizes]
    
    return configured_download_links, configured_download_sizes

configured_download_links, configured_download_sizes = ConfigureDownloadData(download_links, download_sizes, quality, sub_or_dub)
total_download_size = sum(configured_download_sizes)

sub


In [27]:

#Automates download process
def DownloadEpisodes(configured_download_links, download_folder_path):


#Checks whether user is using supported browser then creates a webdriver object of the respective browser and returns it
    def SupportedBrowserCheck():

        edge_options = EdgeOptions()
        chrome_options = ChromeOptions()
        firefox_options = FirefoxOptions()

        edge_options.add_argument("--headless=new")
        chrome_options.add_argument("--headless=new")
        firefox_options.add_argument("--headless=new")

        edge_options.add_experimental_option("prefs", {
            "download.default_directory": download_folder_path,
            "download.prompt_for_download": False,
            "download.directory_upgrade": True,
            "safebrowsing_for_trusted_sources_enabled": False,
            "safebrowsing.enabled": False
        })

        chrome_options.add_experimental_option("prefs", {
            "download.default_directory": download_folder_path,
            "download.prompt_for_download": False,
            "download.directory_upgrade": True,
            "safebrowsing_for_trusted_sources_enabled": False,
            "safebrowsing.enabled": False
        })

    
#Installs Browser Driver Managers for the respective browsers, this is to be used by Selenium
        try:
            service_edge = EdgeService(executable_path=EdgeChromiumDriverManager().install())
            service_chrome = ChromeService(executable_path=ChromeDriverManager().install())
            service_firefox = FirefoxService(executable_path=GeckoDriverManager().install())
        
        except:
            return 0

#Checks whether user is using a supported browser then returns a webdriver object for the respective used browser
        try:
            driver_chrome = webdriver.Chrome(service=service_chrome, options=chrome_options)
            return driver_chrome
        except:
            try:
                driver_edge = webdriver.Edge(service=service_edge, options=edge_options)
                return driver_edge
            except:
                try:
                    driver_firefox = webdriver.Firefox(service=service_firefox, options=firefox_options)
                    print("Setting default anime download location is unsupported in firefox, so check your default browser download location")
                    return driver_firefox
                except:
                    print("Sowwy the onwy supported browses are Chrome, Edge and Firefox")
                    webbrowser.open_new("https://google.com/chrome")

    browser_page = SupportedBrowserCheck()
    
    for configured_download_link in configured_download_links:
        try:
            browser_page.get(configured_download_link)
            time.sleep(6)
            soup = BeautifulSoup(browser_page.page_source, "html.parser")
            server_download_link = soup.find_all("a", class_="btn btn-primary btn-block redirect")[0]["href"]

            browser_page.get(server_download_link)
            
            server_download_link = server_download_link.replace("/f/", "/d/", 1)
            browser_page.find_element(By.CSS_SELECTOR, 'form[action="%s"]' %server_download_link).submit()
        except:
            return 0
    return 1


def DownloadStatus(download_status):
    if download_status:
        return "All downloads completed succesfully, Senpwai ga saikyou no stando Da MUDA"
    elif not download_status:
        return "Error while trying to download, you probably don't have an internet connection, Baka. Please try again uWu"


print(DownloadStatus(DownloadEpisodes(configured_download_links, download_folder_path)))



[WDM] - Downloading: 16.9kB [00:00, 5.78MB/s]                   


All downloads completed succesfully
